In [ ]:
import pandas as pd
import numpy as np
!pip install faker
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker()

N_TOTAL = 50_000
N_COMPLAINTS = 50
N_HISTORY = 1_000

user_ids_normal = random.sample(range(1, 1_000_000), 250_000)
user_ids_bots = random.sample(range(1_000_001, 2_000_000), 25_000) + [0]*25_000
user_ids_all = user_ids_normal + user_ids_bots

good_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/87 Safari/537.36',
    'Mozilla/5.0 (Android 10; Mobile) Firefox/84.0',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 12_4) Safari/604.1'
]

bad_agents = [
    'Python-urllib/3.8', 'Java/1.8.0_261', 'ParserBot/2.1 (+http://example.com/bot)',
    'HeadlessChrome/88.0.4324.96', 'CrawlerBot/4.2 (+http://crawler.example.com)'
]

urls_events = {
    '/auth/login': 1, '/home': 2, '/product/': 3,
    '/cart': 4, '/reviews/': 5, '/payment': 6, '/account': 7
}

domains = ['marketplace.com', 'm.marketplace.com', 'app.ios.marketplace.com', 'app.android.marketplace.com']

start_date = datetime.now() - timedelta(days=1)
datetime_list = [fake.date_time_between(start_date=start_date, end_date='now') for _ in range(N_TOTAL)]

# Основная таблица активности
df_activity = pd.DataFrame({
    'datetime': sorted(datetime_list),
    'ip': [fake.ipv4() for _ in range(N_TOTAL)],
    'user_id': np.random.choice(user_ids_all, N_TOTAL, replace=True),
})

countries = ['Russia', 'USA', 'Germany', 'France', 'India', 'China']
df_activity['country'] = np.random.choice(countries, N_TOTAL, p=[0.85, 0.03, 0.03, 0.03, 0.03, 0.03])

df_activity['net_type'] = np.random.choice(['ok','vpn','proxy','hosting'], N_TOTAL, p=[0.9,0.03,0.04,0.03])

df_activity['user_agent'] = np.random.choice(good_agents*90 + bad_agents*10, N_TOTAL)
df_activity['browser'] = df_activity['user_agent'].apply(lambda ua: 'unknown' if ua in bad_agents else random.choice(['Chrome', 'Firefox', 'Safari']))
df_activity['device'] = df_activity['user_agent'].apply(lambda ua: 'unknown' if ua in bad_agents else random.choice(['Windows', 'Android', 'iOS', 'Linux', 'MacOS']))

df_activity['domain'] = np.random.choice(domains, N_TOTAL)

def gen_url_event():
    url = random.choices(list(urls_events.keys()), weights=[0.2,0.15,0.4,0.05,0.05,0.05,0.1])[0]
    url_full = url + str(random.randint(100,500)) if url in ['/product/', '/reviews/'] else url
    return url_full, urls_events[url]

url_events = [gen_url_event() for _ in range(N_TOTAL)]
df_activity['url'], df_activity['event_id'] = zip(*url_events)

df_activity['session_id'] = df_activity['user_id'].apply(lambda x: fake.uuid4() if x != 0 else np.nan)

# Таблица жалоб
complaint_user_ids = random.sample(user_ids_normal, N_COMPLAINTS)
df_complaints = pd.DataFrame({
    'complaint_date': [fake.date_time_between(start_date=start_date, end_date='now') for _ in range(N_COMPLAINTS)],
    'user_id': complaint_user_ids,
    'email': [fake.email() if random.random() < 0.8 else f"{random.choice(['aaa','zzz','111'])}@{random.choice(['bbb','yyy','222'])}.{random.choice(['com','ru'])}" for _ in range(N_COMPLAINTS)]
})

# Таблица истории пользователей
history_user_ids = random.sample(user_ids_normal+user_ids_bots, N_HISTORY)
df_history = pd.DataFrame({
    'user_id': history_user_ids,
    'change_date': [fake.date_time_between(start_date=start_date, end_date='now') for _ in range(N_HISTORY)],
    'device': [random.choice(['Windows','Android','iOS','Linux','MacOS']) for _ in range(N_HISTORY)]
})

# Объединение таблиц
df_final = df_activity.merge(df_complaints, on='user_id', how='left').merge(df_history, on='user_id', how='left', suffixes=('_complaint', '_history'))

df_final.to_csv('final_dataset.csv', index=False)
df_activity.to_csv('user_activity.csv', index=False)
df_complaints.to_csv('user_complaints.csv', index=False)
df_history.to_csv('user_history.csv', index=False)

In [ ]:
df_final

,datetime,ip,user_id,country,net_type,user_agent,browser,device_complaint,domain,url,event_id,session_id,complaint_date,email,change_date,device_history
0,2025-05-04 22:34:46.575470,70.233.25.157,366735,Russia,ok,Mozilla/5.0 (Android 10; Mobile) Firefox/84.0,Firefox,Windows,app.ios.marketplace.com,/product/399,3,0228ac0c-8d26-4f62-b06a-9fdb39ab1e09,NaT,NaN,NaT,NaN
1,2025-05-04 22:34:46.915708,109.193.27.106,43771,Russia,proxy,Mozilla/5.0 (iPhone; CPU iPhone OS 12_4) Safar...,Firefox,MacOS,app.ios.marketplace.com,/product/182,3,d39dc1a4-e07c-4f05-b8c7-170f85e0b17c,NaT,NaN,NaT,NaN
2,2025-05-04 22:34:48.757644,196.46.61.68,377950,Russia,ok,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chro...,Chrome,Windows,app.android.marketplace.com,/cart,4,3445e3df-4b81-43e6-88ec-ec8dd8cd68dc,NaT,NaN,NaT,NaN
3,2025-05-04 22:34:49.923815,62.167.134.6,509520,Russia,ok,Mozilla/5.0 (Android 10; Mobile) Firefox/84.0,Firefox,Linux,app.android.marketplace.com,/account,7,5e1719bc-3bb5-4852-ba17-6bfc42496c75,NaT,NaN,NaT,NaN
4,2025-05-04 22:34:51.482156,9.45.98.80,0,Russia,ok,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chro...,Chrome,MacOS,marketplace.com,/account,7,NaN,NaT,NaN,2025-05-05 06:58:12.093108,iOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397347,2025-05-05 22:34:35.766473,33.143.236.39,0,Russia,ok,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chro...,Chrome,MacOS,app.ios.marketplace.com,/cart,4,NaN,NaT,NaN,2025-05-05 08:02:02.067132,MacOS
397348,2025-05-05 22:34:35.766473,33.143.236.39,0,Russia,ok,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chro...,Chrome,MacOS,app.ios.marketplace.com,/cart,4,NaN,NaT,NaN,2025-05-05 14:02:38.279513,MacOS
397349,2025-05-05 22:34:38.198575,164.6.44.116,628957,Russia,ok,Mozilla/5.0 (Android 10; Mobile) Firefox/84.0,Chrome,Android,app.ios.marketplace.com,/product/253,3,ee261d77-d1b4-4203-ac01-7ebc0adcb01c,NaT,NaN,NaT,NaN
397350,2025-05-05 22:34:39.986463,168.91.202.196,531657,Russia,vpn,ParserBot/2.1 (+http://example.com/bot),unknown,unknown,app.android.marketplace.com,/product/497,3,daed4e99-7125-47fa-ae91-f6d100010f4c,NaT,NaN,NaT,NaN


In [ ]:
import re
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv('final_dataset.csv', low_memory=False)

df.drop('datetime', axis=1, inplace=True)

def expanding_nunique(series):
    seen = set()
    counts = []
    for val in series:
        seen.add(val)
        counts.append(len(seen))
    return counts

df['user_unique_ip_count'] = df.groupby('user_id')['ip'].transform(expanding_nunique)

df['is_authorized'] = (df['user_id'] != 0).astype(int)
df['user_status'] = (df['user_id'] <= 1_000_000).astype(int)

suspicious_keywords = ['Python', 'Java', 'Bot', 'Crawler', 'Headless', 'Parser', 'urllib']
pattern = '|'.join(suspicious_keywords)
df['is_suspicious_agent'] = df['user_agent'].str.contains(pattern, case=False, regex=True).astype(int)

event_dummies = pd.get_dummies(df['event_id'], prefix='event_id_freq')
event_freq = event_dummies.groupby(df['user_id']).cumsum()
df = pd.concat([df, event_freq], axis=1)

df['session_is_authorized'] = (~df['session_id'].isna()).astype(int)

def expanding_auth_ratio(series):
    auth_count = 0
    ratios = []
    for idx, val in enumerate(series, start=1):
        auth_count += val
        ratios.append(auth_count / idx)
    return ratios

df['auth_session_ratio'] = df.groupby('user_id')['session_is_authorized'].transform(expanding_auth_ratio)

def vowel_ratio(s):
    vowels = re.findall(r'[aeiou]', s.lower())
    return len(vowels) / len(s) if len(s) > 0 else 0

def consonant_ratio(s):
    consonants = re.findall(r'[bcdfghjklmnpqrstvwxyz]', s.lower())
    return len(consonants) / len(s) if len(s) > 0 else 0

def digit_ratio(s):
    digits = re.findall(r'\d', s)
    return len(digits) / len(s) if len(s) > 0 else 0

def specialchar_ratio(s):
    specialchars = re.findall(r'[^a-zA-Z0-9@.]', s)
    return len(specialchars) / len(s) if len(s) > 0 else 0

df['has_complaint'] = (~df['email'].isna()).astype(int)
df['email_length'] = df['email'].fillna('').apply(len)
df['email_unique_symbols'] = df['email'].fillna('').apply(lambda x: len(set(x)))
df['email_name_length'] = df['email'].fillna('').apply(lambda x: len(x.split('@')[0]))
df['email_vowel_ratio'] = df['email'].fillna('').apply(vowel_ratio)
df['email_consonant_ratio'] = df['email'].fillna('').apply(consonant_ratio)
df['email_digit_ratio'] = df['email'].fillna('').apply(digit_ratio)
df['email_specialchar_ratio'] = df['email'].fillna('').apply(specialchar_ratio)

df['device_mismatch'] = ((df['device_history'].notna()) & (df['device_complaint'] != df['device_history'])).astype(int)

df['user_unique_device_count'] = df.groupby('user_id')['device_history'].transform(expanding_nunique)

cols_to_drop = ['ip', 'user_id', 'session_id', 'complaint_date', 'email', 'change_date', 'device_history', 'user_agent', 'url', 'event_id']
df.drop(cols_to_drop, axis=1, inplace=True)

ohe_cols = ['country', 'net_type', 'browser', 'device_complaint', 'domain']
df = pd.get_dummies(df, columns=ohe_cols, dummy_na=False)

df.fillna(0, inplace=True)

numeric_cols = [
    'user_unique_ip_count', 'auth_session_ratio', 'email_length', 'email_unique_symbols',
    'email_name_length', 'email_vowel_ratio', 'email_consonant_ratio',
    'email_digit_ratio', 'email_specialchar_ratio', 'user_unique_device_count'
]

scaler = MinMaxScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

df.to_csv('preprocessed_final_dataset.csv', index=False)
df

,user_unique_ip_count,is_authorized,user_status,is_suspicious_agent,event_id_freq_1,event_id_freq_2,event_id_freq_3,event_id_freq_4,event_id_freq_5,event_id_freq_6,...,device_complaint_Android,device_complaint_Linux,device_complaint_MacOS,device_complaint_Windows,device_complaint_iOS,device_complaint_unknown,domain_app.android.marketplace.com,domain_app.ios.marketplace.com,domain_m.marketplace.com,domain_marketplace.com
0,0.0,1,1,0,0,0,1,0,0,0,...,False,False,False,True,False,False,False,True,False,False
1,0.0,1,1,0,0,0,1,0,0,0,...,False,False,True,False,False,False,False,True,False,False
2,0.0,1,1,0,0,0,0,1,0,0,...,False,False,False,True,False,False,True,False,False,False
3,0.0,1,1,0,0,0,0,0,0,0,...,False,True,False,False,False,False,True,False,False,False
4,0.0,0,1,0,0,0,0,0,0,0,...,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397347,1.0,0,1,0,67728,55361,143673,18591,14857,14940,...,False,False,True,False,False,False,False,True,False,False
397348,1.0,0,1,0,67728,55361,143673,18592,14857,14940,...,False,False,True,False,False,False,False,True,False,False
397349,0.0,1,1,0,0,0,1,0,0,0,...,True,False,False,False,False,False,False,True,False,False
397350,0.0,1,1,1,0,0,1,0,0,0,...,False,False,False,False,False,True,True,False,False,False
